# Optimization - fine tuning

[1. Optimize LSH](#1-optimize-lsh)

[2. Optimize Hypercube](#2-optimize-hypercube)

[3. Optimize GNNS](#3-optimize-gnns)

[4. Optimize MRNG](#4-optimize-mrng)

# Import libraries

In [1]:
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

import pandas

from params import lsh_test, hypercube_test, gnn_test, mrng_test

In [2]:
input_path = b'../MNIST/input.dat'
query_path = b'../MNIST/query.dat'

n = 60000

# 1. Optimize LSH

## Using Botorch sampler

To skip logs, click [here](#visualize-lsh-study-results-botorch).

In [3]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size':  trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 100, 5000),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False])
                 }
    
    print("Trial parameters:", param_dict)

    average_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=100, **param_dict, N=60)

    # trial should return at least 60 neighbors to be used in GNNS
    # penalize model if slower than brute force
    c0 = - min_neighbors.value + 60
    c1 = average_time.value - 0.01
    trial.set_user_attr('constraint', (c0, c1))

    return maf.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [4]:
%%time
for i in range(10):
    try:
        sampler = optuna.integration.BoTorchSampler(constraints_func=constraints, n_startup_trials=10)
        lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
        lsh_study.optimize(objective_lsh, n_trials=50)
        print("-------------------- Best trials --------------------")
        trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
        # print feasible trials only
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

<timed exec>:3: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
[I 2023-11-26 10:21:59,492] A new study created in memory with name: lsh


Trial parameters: {'k': 3, 'L': 7, 'table_size': 7500, 'window_size': 1458, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 

[I 2023-11-26 10:22:37,003] Trial 0 finished with values: [1.0991609144206673, 0.054653260000000016] and parameters: {'k': 3, 'L': 7, 'table_size': 7500, 'window_size': 1458, 'query_trick': True}. 


Trial parameters: {'k': 5, 'L': 7, 'table_size': 7500, 'window_size': 2616, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 10:23:23,531] Trial 1 finished with values: [1.0316464170281046, 0.08851505000000003] and parameters: {'k': 5, 'L': 7, 'table_size': 7500, 'window_size': 2616, 'query_trick': False}. 


...

Trial parameters: {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3466, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 12:26:18,381] Trial 48 finished with values: [1.0450836773108385, 0.04061297] and parameters: {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3466, 'query_trick': True}. 
/home/elvrach/.local/lib/python3.10/site-packages/optuna/integration/botorch.py:843: ExperimentalWarning: qehvi_candidates_func is experimental (supported from v2.4.0). The interface can change in the future.
  candidates = self._candidates_func(


Trial parameters: {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3431, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 12:27:52,840] Trial 49 finished with values: [1.0262989668451132, 0.03300241000000002] and parameters: {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3431, 'query_trick': True}. 


-------------------- Best trials --------------------
Trial no. 3
 Values = [1.0, 0.32503074999999987], Constraints = (0, 0.31503074999999986)
 Params = {'k': 3, 'L': 5, 'table_size': 15000, 'window_size': 3231, 'query_trick': True}
Trial no. 45
 Values = [1.019653796157214, 0.05082782999999999], Constraints = (0, 0.04082782999999999)
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3689, 'query_trick': True}
Trial no. 49
 Values = [1.0262989668451132, 0.03300241000000002], Constraints = (0, 0.023002410000000015)
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3431, 'query_trick': True}
Trial no. 47
 Values = [1.0394867421483158, 0.032416919999999995], Constraints = (0, 0.022416919999999993)
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3349, 'query_trick': True}
Trial no. 32
 Values = [1.0673949764669912, 0.026533179999999996], Constraints = (0, 0.016533179999999995)
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 3057, 

### Visualize LSH study results (Botorch)

In [5]:
plot_pareto_front(lsh_study, target_names=['maf', 'average_time'])

In [6]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [7]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [8]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [9]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

## Using NSGAII sampler

To skip logs, click [here](#visualize-lsh-study-results-nsgaii).

In [ ]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False]),
                 }
    
    print("Trial parameters:", param_dict)

    average_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=100, **param_dict, N=60)

    # trial should return at least 60 neighbors to be used in GNNS
    # penalize model if slower than brute force
    c0 = - min_neighbors.value + 60
    c1 = average_time.value - 0.01
    trial.set_user_attr('constraint', (c0, c1))

    return maf.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [ ]:
%%time
for i in range(10):
    try:
        sampler = optuna.samplers.NSGAIISampler(constraints_func=constraints)
        lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
        lsh_study.optimize(objective_lsh, n_trials=50)
        print("-------------------- Best trials --------------------")
        trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
        # print feasible trials only
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

/home/thantrisp/miniconda3/envs/testpy/lib/python3.12/site-packages/optuna/samplers/nsgaii/_sampler.py:162: ExperimentalWarning: The constraints_func option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-11-26 16:14:04,282] A new study created in memory with name: lsh


Trial parameters: {'k': 10, 'L': 6, 'table_size': 15000, 'window_size': 2190, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 16:14:52,662] Trial 0 finished with values: [1.4737879905085032, 0.0029157] and parameters: {'k': 10, 'L': 6, 'table_size': 15000, 'window_size': 2190, 'query_trick': True}. 


Trial parameters: {'k': 3, 'L': 3, 'table_size': 3750, 'window_size': 1063, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 16:15:23,545] Trial 1 finished with values: [1.499912106564422, 0.00921434] and parameters: {'k': 3, 'L': 3, 'table_size': 3750, 'window_size': 1063, 'query_trick': False}. 


...

Trial parameters: {'k': 8, 'L': 9, 'table_size': 15000, 'window_size': 2242, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 16:55:00,642] Trial 48 finished with values: [1.152395647747845, 0.010039130000000002] and parameters: {'k': 8, 'L': 9, 'table_size': 15000, 'window_size': 2242, 'query_trick': False}. 


Trial parameters: {'k': 3, 'L': 5, 'table_size': 7500, 'window_size': 2881, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 

[I 2023-11-26 16:55:50,453] Trial 49 finished with values: [1.0152371815700856, 0.14100393999999997] and parameters: {'k': 3, 'L': 5, 'table_size': 7500, 'window_size': 2881, 'query_trick': True}. 


-------------------- Best trials --------------------
Trial no. 31
 Values = [1.0, 0.15069452999999997], Constraints = (0, 0.14069452999999996)
 Params = {'k': 6, 'L': 10, 'table_size': 3750, 'window_size': 3677, 'query_trick': True}
Trial no. 2
 Values = [1.0016436954345849, 0.12031295999999997], Constraints = (0, 0.11031295999999997)
 Params = {'k': 4, 'L': 8, 'table_size': 3750, 'window_size': 2921, 'query_trick': True}
Trial no. 20
 Values = [1.0019077119803148, 0.09559584999999998], Constraints = (0, 0.08559584999999999)
 Params = {'k': 5, 'L': 7, 'table_size': 3750, 'window_size': 2805, 'query_trick': True}
Trial no. 9
 Values = [1.01098321046841, 0.07159987000000002], Constraints = (0, 0.06159987000000002)
 Params = {'k': 9, 'L': 6, 'table_size': 7500, 'window_size': 4659, 'query_trick': True}
Trial no. 47
 Values = [1.0157488177897585, 0.06259387], Constraints = (0, 0.052593869999999994)
 Params = {'k': 7, 'L': 8, 'table_size': 3750, 'window_size': 3304, 'query_trick': False}
T

### Visualize LSH study results (NSGAII)

In [ ]:
plot_pareto_front(lsh_study, target_names=['MAF', 'Average time'])

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'MAF')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'Average time')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'MAF')

## Optimization without constraints

To skip logs, click [here](#visualize-lsh-study-results-without-constraints).

In [ ]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False])
                 }
    
    print("Trial parameters:", param_dict)

    average_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=100, **param_dict, N=50)

    return maf.value, average_time.value, min_neighbors.value

In [ ]:
%%time
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize', 'maximize'])
lsh_study.optimize(objective_lsh, n_trials=50)

[I 2023-11-26 18:29:54,478] A new study created in memory with name: lsh


Trial parameters: {'k': 9, 'L': 6, 'table_size': 15000, 'window_size': 2456, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 18:30:38,632] Trial 0 finished with values: [1.2863161135729677, 0.004823560000000001, 50.0] and parameters: {'k': 9, 'L': 6, 'table_size': 15000, 'window_size': 2456, 'query_trick': True}. 


Trial parameters: {'k': 2, 'L': 7, 'table_size': 3750, 'window_size': 1288, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 18:31:17,262] Trial 1 finished with values: [1.027435474263356, 0.07817652000000001, 50.0] and parameters: {'k': 2, 'L': 7, 'table_size': 3750, 'window_size': 1288, 'query_trick': False}. 


...

Trial parameters: {'k': 5, 'L': 9, 'table_size': 15000, 'window_size': 3438, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 19:13:49,792] Trial 48 finished with values: [1.0, 0.22487005000000007, 50.0] and parameters: {'k': 5, 'L': 9, 'table_size': 15000, 'window_size': 3438, 'query_trick': False}. 


Trial parameters: {'k': 3, 'L': 4, 'table_size': 15000, 'window_size': 3182, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 19:14:44,072] Trial 49 finished with values: [1.003079571761728, 0.14983422000000002, 50.0] and parameters: {'k': 3, 'L': 4, 'table_size': 15000, 'window_size': 3182, 'query_trick': True}. 


CPU times: user 34min 5s, sys: 10min 39s, total: 44min 44s
Wall time: 44min 49s


In [ ]:
print("-------------------- Best trials --------------------")
trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
# print feasible trials only
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))


-------------------- Best trials --------------------
Trial no. 18
 Values = [1.0, 0.13765656999999992, 50.0]
 Params = {'k': 8, 'L': 10, 'table_size': 7500, 'window_size': 4570, 'query_trick': True}
Trial no. 23
 Values = [1.0000329705767328, 0.07704273000000002, 50.0]
 Params = {'k': 10, 'L': 9, 'table_size': 15000, 'window_size': 4428, 'query_trick': False}
Trial no. 13
 Values = [1.0017920583849802, 0.07380007999999999, 50.0]
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 4989, 'query_trick': False}
Trial no. 29
 Values = [1.0064591950958244, 0.061548800000000015, 50.0]
 Params = {'k': 7, 'L': 9, 'table_size': 3750, 'window_size': 3629, 'query_trick': False}
Trial no. 11
 Values = [1.014306164419062, 0.05221652000000002, 50.0]
 Params = {'k': 8, 'L': 10, 'table_size': 7500, 'window_size': 3824, 'query_trick': True}
Trial no. 36
 Values = [1.0309397645013207, 0.03882437999999999, 50.0]
 Params = {'k': 7, 'L': 10, 'table_size': 15000, 'window_size': 2657, 'query_tric

### Visualize LSH study results (without constraints)

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[2], target_name = 'min_neighbors')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[2], target_name = 'min_neighbors')

# 2. Optimize Hypercube

To skip logs, click [here](#visualize-hypercube-study-results).

In [12]:
def objective_hypercube(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 30),
                  'M': trial.suggest_int('M', 10, 5000),
                  'probes': trial.suggest_int('probes', 1, 1000)
                 }
    
    average_time, maf = hypercube_test(input_path, query_path, queries_num=10, **param_dict, N=5)

    return maf.value, average_time.value

In [13]:
%%time
hypercube_study = optuna.create_study(study_name='hypercube', directions=['minimize', 'minimize'])
hypercube_study.optimize(objective_hypercube, n_trials=50)
print("-----------------------------------------------------")

trials = sorted(hypercube_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2023-11-26 13:14:13,296] A new study created in memory with name: hypercube


Read MNIST data
Preprocessing...
Preprocessing time: 3.08277
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-26 13:14:19,947] Trial 0 finished with values: [1.3052526703808836, 0.0042566] and parameters: {'k': 13, 'M': 4284, 'probes': 836}. 


Read MNIST data
Preprocessing...
Preprocessing time: 4.85208
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-26 13:14:28,550] Trial 1 finished with values: [1.3223780947068637, 0.0155388] and parameters: {'k': 20, 'M': 1243, 'probes': 430}. 


...

Read MNIST data
Preprocessing...
Preprocessing time: 6.65548
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-26 13:20:58,487] Trial 48 finished with values: [1.5765342728732308, 0.12237039999999999] and parameters: {'k': 25, 'M': 496, 'probes': 790}. 


Read MNIST data
Preprocessing...
Preprocessing time: 0.805071
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-26 13:21:03,444] Trial 49 finished with values: [2.081984137232106, 0.0008331999999999999] and parameters: {'k': 3, 'M': 3545, 'probes': 175}. 


-----------------------------------------------------
Trial no. 31
 Values = [1.2543074986697542, 0.0103444]
 Params = {'k': 15, 'M': 49, 'probes': 502}
Trial no. 0
 Values = [1.3052526703808836, 0.0042566]
 Params = {'k': 13, 'M': 4284, 'probes': 836}
Trial no. 28
 Values = [1.529160171998507, 0.0037799]
 Params = {'k': 14, 'M': 3535, 'probes': 495}
Trial no. 10
 Values = [1.5802614066163017, 0.0019048]
 Params = {'k': 11, 'M': 2443, 'probes': 323}
Trial no. 5
 Values = [1.652224176007537, 0.0015421999999999999]
 Params = {'k': 6, 'M': 1737, 'probes': 459}
Trial no. 4
 Values = [1.920949270974015, 0.0011194999999999998]
 Params = {'k': 7, 'M': 3204, 'probes': 273}
Trial no. 49
 Values = [2.081984137232106, 0.0008331999999999999]
 Params = {'k': 3, 'M': 3545, 'probes': 175}
Trial no. 19
 Values = [2.462393343172151, 0.000815]
 Params = {'k': 7, 'M': 4623, 'probes': 119}
CPU times: user 5min 50s, sys: 1min, total: 6min 50s
Wall time: 6min 50s


## Visualize Hypercube study results

In [18]:
plot_pareto_front(hypercube_study, target_names=['maf', 'average_time'])

In [19]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[0], target_name = 'maf')

In [20]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

In [21]:
plot_slice(hypercube_study, target = lambda t: t.values[0], target_name = 'maf')

In [22]:
plot_slice(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

# 3. Optimize GNNS

To skip logs, click [here](#visualize-gnns-study-results).

## Visualize GNNS study results

In [3]:
def objective_gnns(trial):
    param_dict = {'k': trial.suggest_int('k', 40, 100)}
    param_dict.update({'E': trial.suggest_int('E', 40, param_dict['k'])})
    param_dict.update({'R': trial.suggest_int('R', 1, 10)})
    param_dict.update({'N': trial.suggest_int('N', 1, 10)})

    print("Trial params", param_dict)

    average_time, maf = gnn_test(input_path, query_path, queries_num=100, **param_dict)

    return maf.value, average_time.value

In [4]:
%%time
for i in range(10):
    try:
        gnns_study = optuna.create_study(study_name='gnns', directions=['minimize', 'minimize'])
        gnns_study.optimize(objective_gnns, n_trials=50)
        print("-------------------- Best trials --------------------")
        trials = sorted(gnns_study.best_trials, key=lambda x: x.values)
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}".format(trial.values))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

[I 2023-11-26 20:24:08,310] A new study created in memory with name: gnns


Trial params {'k': 50, 'E': 46, 'R': 4, 'N': 2}
Read MNIST data
Building graph...
LSH initialization time: 11.0035
GNN initialization time: 215.392
Done
Algorithm: GNN
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Qu

[I 2023-11-26 20:28:19,031] Trial 0 finished with values: [1.9563989255370688, 0.0023223099999999993] and parameters: {'k': 50, 'E': 46, 'R': 4, 'N': 2}. 


Trial params {'k': 98, 'E': 89, 'R': 8, 'N': 7}
Read MNIST data
Building graph...
LSH initialization time: 11.0969
GNN initialization time: 149.054
Done
Algorithm: GNN
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Qu

[I 2023-11-26 20:31:22,412] Trial 1 finished with values: [1.3995281678630607, 0.00632105] and parameters: {'k': 98, 'E': 89, 'R': 8, 'N': 7}. 


...

Trial params {'k': 64, 'E': 50, 'R': 4, 'N': 5}
Read MNIST data
Building graph...
LSH initialization time: 11.23
GNN initialization time: 217.159
Done
Algorithm: GNN
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Quer

[I 2023-11-26 23:01:27,893] Trial 48 finished with values: [1.5009220102342202, 0.0025843199999999998] and parameters: {'k': 64, 'E': 50, 'R': 4, 'N': 5}. 


Trial params {'k': 46, 'E': 44, 'R': 4, 'N': 8}
Read MNIST data
Building graph...
LSH initialization time: 11.216
GNN initialization time: 194.691
Done
Algorithm: GNN
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Que

[I 2023-11-26 23:05:18,334] Trial 49 finished with values: [1.4961082345456551, 0.002298849999999998] and parameters: {'k': 46, 'E': 44, 'R': 4, 'N': 8}. 


-------------------- Best trials --------------------
Trial no. 6
 Values = [1.1097230214820741, 0.007753779999999996]
 Params = {'k': 87, 'E': 84, 'R': 9, 'N': 10}
Trial no. 43
 Values = [1.1681959950659913, 0.00629306]
 Params = {'k': 64, 'E': 61, 'R': 10, 'N': 3}
Trial no. 15
 Values = [1.2573720949263423, 0.00515043]
 Params = {'k': 98, 'E': 78, 'R': 7, 'N': 7}
Trial no. 12
 Values = [1.3708144942295668, 0.0031303100000000007]
 Params = {'k': 70, 'E': 56, 'R': 5, 'N': 9}
Trial no. 30
 Values = [1.3939309482239626, 0.002817849999999999]
 Params = {'k': 88, 'E': 78, 'R': 3, 'N': 6}
Trial no. 49
 Values = [1.4961082345456551, 0.002298849999999998]
 Params = {'k': 46, 'E': 44, 'R': 4, 'N': 8}
Trial no. 3
 Values = [1.6349565199168101, 0.0020252599999999996]
 Params = {'k': 97, 'E': 41, 'R': 4, 'N': 7}
Trial no. 28
 Values = [1.793904840976792, 0.0017898200000000001]
 Params = {'k': 49, 'E': 44, 'R': 3, 'N': 6}
Trial no. 24
 Values = [2.0090560527721957, 0.0012146299999999997]
 Params =

In [10]:
plot_pareto_front(gnns_study, target_names=['maf', 'average_time'])

In [11]:
plot_optimization_history(gnns_study, target = lambda t: t.values[0], target_name = 'maf')

In [12]:
plot_optimization_history(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

In [13]:
plot_slice(gnns_study, target = lambda t: t.values[0], target_name = 'maf')

In [14]:
plot_slice(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

# 4. Optimize MRNG

To skip logs, click [here](#visualize-mrng-study-results).

In [7]:
def objective_mrng(trial):
    param_dict = {'l': trial.suggest_int('l', 1, 1000)}
    param_dict.update({'N': trial.suggest_int('N', 1, param_dict['l'])})
    
    print("Trial parameters:", param_dict)

    average_time, maf = mrng_test(input_path, query_path, queries_num=100, **param_dict)

    return maf.value, average_time.value

In [8]:
%%time
for i in range(10):
    try:
        mrng_study = optuna.create_study(study_name='mrng', directions=['minimize', 'minimize'])
        mrng_study.optimize(objective_mrng, n_trials=50)
        print("-------------------- Best trials --------------------")
        trials = sorted(mrng_study.best_trials, key=lambda x: x.values)
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}".format(trial.values))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

[I 2023-11-27 10:01:35,585] A new study created in memory with name: mrng


Trial parameters: {'l': 589, 'N': 374}
Read MNIST data
Loading graph from file: ../graph_files/mrng_graph.bin
Done
Algorithm: MRNG
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 87

[I 2023-11-27 10:02:01,987] Trial 0 finished with values: [1.0041997001402891, 0.0020086099999999996] and parameters: {'l': 589, 'N': 374}. 


Trial parameters: {'l': 685, 'N': 273}
Read MNIST data
Loading graph from file: ../graph_files/mrng_graph.bin
Done
Algorithm: MRNG
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 87

[I 2023-11-27 10:02:28,252] Trial 1 finished with values: [1.0041997001402891, 0.0022238399999999995] and parameters: {'l': 685, 'N': 273}. 


...

Trial parameters: {'l': 959, 'N': 878}
Read MNIST data
Loading graph from file: ../graph_files/mrng_graph.bin
Done
Algorithm: MRNG
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 87

[I 2023-11-27 10:21:40,305] Trial 48 finished with values: [1.0, 0.0023659600000000003] and parameters: {'l': 959, 'N': 878}. 


Trial parameters: {'l': 527, 'N': 116}
Read MNIST data
Loading graph from file: ../graph_files/mrng_graph.bin
Done
Algorithm: MRNG
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 87

[I 2023-11-27 10:22:03,785] Trial 49 finished with values: [1.0152371815700856, 0.0015190499999999997] and parameters: {'l': 527, 'N': 116}. 


-------------------- Best trials --------------------
Trial no. 32
 Values = [1.0, 0.00221023]
 Params = {'l': 894, 'N': 765}
Trial no. 34
 Values = [1.0001328020386833, 0.0021092699999999995]
 Params = {'l': 793, 'N': 442}
Trial no. 31
 Values = [1.00069342713173, 0.00193085]
 Params = {'l': 753, 'N': 187}
Trial no. 42
 Values = [1.0041997001402891, 0.0016173699999999997]
 Params = {'l': 594, 'N': 465}
Trial no. 49
 Values = [1.0152371815700856, 0.0015190499999999997]
 Params = {'l': 527, 'N': 116}
Trial no. 27
 Values = [1.028446590467221, 0.0013056]
 Params = {'l': 487, 'N': 315}
Trial no. 24
 Values = [1.0394867421483158, 0.0011740399999999997]
 Params = {'l': 414, 'N': 184}
Trial no. 25
 Values = [1.0589503757518537, 0.0011199400000000007]
 Params = {'l': 398, 'N': 177}
Trial no. 26
 Values = [1.1663822636293262, 0.0010406699999999998]
 Params = {'l': 348, 'N': 268}
Trial no. 36
 Values = [1.2971186638513006, 0.00102835]
 Params = {'l': 299, 'N': 99}
Trial no. 19
 Values = [2.1145

## Visualize MRNG study results

In [9]:
plot_pareto_front(mrng_study, target_names=['maf', 'average_time'])

In [10]:
plot_optimization_history(mrng_study, target = lambda t: t.values[0], target_name = 'maf')

In [11]:
plot_optimization_history(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')

In [12]:
plot_slice(mrng_study, target = lambda t: t.values[0], target_name = 'maf')

In [13]:
plot_slice(mrng_study, target = lambda t: t.values[1], target_name = 'average_time')